In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.callback.tensorboard import *

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import datetime
import pandas as pd
import numpy as np

In [5]:
from fastai.callback.fp16 import *

In [6]:
cuda0 = torch.device('cuda:0')
cpu = torch.device('cpu')

In [7]:
def get_y(r): return parent_label(r).split(" ")

In [8]:
from fastai.losses import *
from torch.nn.modules import loss

class smooth_binary_cross_entropy(loss._Loss):

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean',
             pos_weight: Optional[Tensor] = None) -> None:
        super(smooth_binary_cross_entropy, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.register_buffer('pos_weight', pos_weight)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        c = target.shape[1]
        eps = 0.1
        smoothed_target = torch.where(target==1, 1-(eps+(eps/c)), eps/c)
        return F.binary_cross_entropy_with_logits(input,
                                                  smoothed_target,
                                                  pos_weight=self.pos_weight,
                                                  reduction=self.reduction)

@delegates()
class smooth_loss_v2(BaseLoss):
    "Same as `nn.BCEWithLogitsLoss`, but flattens input and target."
    @use_kwargs_dict(keep=True, weight=None, reduction='mean', pos_weight=None)
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        if kwargs.get('pos_weight', None) is not None and kwargs.get('flatten', None) is True:
            raise ValueError("`flatten` must be False when using `pos_weight` to avoid a RuntimeError due to shape mismatch")
        if kwargs.get('pos_weight', None) is not None: kwargs['flatten'] = False
        super().__init__(smooth_binary_cross_entropy, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

In [9]:
path = Path("../data/test")

In [10]:
images = get_image_files(path)
images

(#610) [Path('../data/test/008FWT.JPG'),Path('../data/test/00AQXY.JPG'),Path('../data/test/01OJZX.JPG'),Path('../data/test/07OXKK.jfif'),Path('../data/test/085IEC.jpg'),Path('../data/test/08O2YE.JPG'),Path('../data/test/08WLJO.jfif'),Path('../data/test/0E1VTP.jfif'),Path('../data/test/0GJFRQ.JPG'),Path('../data/test/0J3PQ7.JPG')...]

In [11]:
models_dir = Path('../models/exported')
learn_inf = load_learner(models_dir/"125719_resnet50_B64S460.pkl", cpu=False)

In [12]:
test_dl = learn_inf.dls.test_dl(images)

In [13]:
with learn_inf.no_bar():
    predictions, _ = learn_inf.tta(dl=test_dl)

In [14]:
len(predictions)

610

In [15]:
image_ids = [image.name.split(".")[0] for image in images]
preds = [[leaf_rust.item(), stem_rust.item(), healthy_wheat.item()] for healthy_wheat, stem_rust, leaf_rust in predictions]
df = pd.DataFrame(preds, columns=('leaf_rust', 'stem_rust', 'healthy_wheat'), index=image_ids)
df.index.name = "ID"

In [16]:
df = df.round(decimals=2)

In [17]:
df.to_csv(Path("125719_resnet50_B64S460.csv"))

In [18]:
df

,leaf_rust,stem_rust,healthy_wheat
ID,,,
008FWT,0.19,0.67,0.20
00AQXY,0.90,0.06,0.07
01OJZX,0.28,0.65,0.16
07OXKK,0.22,0.22,0.63
085IEC,0.23,0.06,0.80
...,...,...,...
ZKF0L9,0.12,0.62,0.14
ZMAB3R,0.83,0.13,0.11
ZOXDKA,0.86,0.07,0.04
